In [1]:
from keras.models import Sequential

Using TensorFlow backend.


In [3]:
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [11]:
import pandas as pd
import numpy as np
import os 
from sklearn.preprocessing import MinMaxScaler
os.chdir("C:\Python Trainer\mlarmy\Projects\Google Stock Price Prediction using RNN\google-stock-price")

In [7]:
df_train = pd.read_csv("Google_Stock_Price_Train.csv")
df_train.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"


In [10]:
train_set = df_train.iloc[:,1:2].values

In [13]:
sc = MinMaxScaler(feature_range=(0,1))
train_set_scaled = sc.fit_transform(train_set)
train_set_scaled

array([[ 0.08581368],
       [ 0.09701243],
       [ 0.09433366],
       ..., 
       [ 0.95725128],
       [ 0.93796041],
       [ 0.93688146]])

In [20]:
#creating a datastructure with 60 timesteps and 1 output 
X_train = []
y_train = []
for i in range(60,1258):
    X_train.append(train_set_scaled[i-60:i, 0])
    y_train.append(train_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
print(X_train.shape)
print(y_train.shape)

#Reshaping 
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
print("New shape: ", X_train.shape)

(1198, 60)
(1198,)
New shape:  (1198, 60, 1)


In [22]:
#Initialize the RNN
regressor = Sequential()

In [23]:
#Adding the 1st RNN LSTM layer and dropout regularization 
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

In [26]:
#Adding the 2nd LSTM layer and dropout regularization
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

In [27]:
#Adding the 3rd LSTM layer and dropout regularization
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

In [28]:
#Adding the 4th LSTM layer and dropout regularization
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

In [29]:
#Adding the output layer
regressor.add(Dense(units = 1))

In [30]:
#Compile the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [31]:
#Fitting the RNN to the training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
1198/1198 [==============================] - 28s 23ms/step - loss: 0.0413

KeyboardInterrupt: 

In [40]:
#Load the test data 
df_test = pd.read_csv("Google_Stock_Price_Test.csv")
df_test.head()
real_stock_price = df_test.iloc[:, 1:2].values
real_stock_price

array([[ 778.81],
       [ 788.36],
       [ 786.08],
       [ 795.26],
       [ 806.4 ],
       [ 807.86],
       [ 805.  ],
       [ 807.14],
       [ 807.48],
       [ 807.08],
       [ 805.81],
       [ 805.12],
       [ 806.91],
       [ 807.25],
       [ 822.3 ],
       [ 829.62],
       [ 837.81],
       [ 834.71],
       [ 814.66],
       [ 796.86]])

In [47]:
#Prepare test set
dataset_total = pd.concat((df_train["Open"], df_test["Open"]), axis = 0)
inputs = dataset_total[len(dataset_total) - len(df_test) - 60:].values
inputs = inputs.reshape(-1, 1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

#Predict RNN model on the test data
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)
predicted_stock_price

array([[ 757.00421143],
       [ 758.0713501 ],
       [ 758.91455078],
       [ 759.54937744],
       [ 760.02404785],
       [ 760.4208374 ],
       [ 760.82159424],
       [ 761.28186035],
       [ 761.83789062],
       [ 762.50646973],
       [ 763.28564453],
       [ 764.15686035],
       [ 765.09155273],
       [ 766.06268311],
       [ 767.04602051],
       [ 768.05841064],
       [ 769.14080811],
       [ 770.34838867],
       [ 771.71246338],
       [ 773.18774414]], dtype=float32)